# Exercise 3 - SAKI

The purpose of this exercise is to develop an algorithm that optimizes the route that a robot takes for pick-up and storage of items in a warehouse.

There are the following constraints : 

* Size of warehouse is {1..3} x {1..3}
* There is separate start/stop position outside the 3x3 storage space
* The first position the robot can move into is always (1, 1)
* Robots can move to adjacent fields (but not diagonally)
* There are three types of items, identified by color (white, blue, red)


Here are the main lines of our work plan
* I. Introduction 

* II. Data Analysis / Data Extraction

* III. Markov Decision Process

* IV. Testing

* V. Evaluation

# I. Introduction

Reinforcement learning (RL) is learning what to do, how to map situations to actions, so as to maximize
a numerical reward signal. The learner is not told which actions to take, but instead must discover
which actions yield the most reward by trying them.

In short, we can say that reinforcement learning is learning by trying.

In RL there is no supervisor, only a reward signal or a real number that tells the agent how good or bad was his action. 
Feedback from the environment might be delayed over several time steps, it’s not necessarily instantaneous 
e.g. for the task of reaching a goal in a grid-world, the feedback might be at the end when the agent reaches the goal. 
The agent might spend some time exploring and wandering in the environment until it finally reaches the goal after a while to realize what were the good and bad actions it has taken.

In machine learning or supervised learning, we have a dataset that describes the environment to the algorithm and the right answers or actions to do when faced with a specific situation, and the algorithm tries to generalize from that data to new situations.

In RL the agent influences the environment through its actions which in turn affect the subsequent data it receives from the environment, it’s an active learning process.

In the problem, an agent is supposed to decide the best action to select based on his current state. When this step is repeated, the problem is known as a __Markov Decision Process__.


In [11]:
# We start by importing all the librairies necessary for ourn work.
import csv
from itertools import product
import mdptoolbox

import numpy as np
import pandas as pd

# II. Data Analysis / Data Extraction

Before defining the different parameters of the Markov Decision Process, we will start by importing our different data for analysis. 

We have 2 files at our disposal representing the training set and the testing set. 



In [2]:
# import file training and testing file into a variable

training_data_set = pd.read_csv("data/SAKI Exercise 3 warehousetraining2x2.txt", delimiter='\t', names=["action", "color"])
testing_data_set = pd.read_csv("data/SAKI Exercise 3 warehouseorder2x2.txt", delimiter='\t', names=["action", "color"])

print('Training Set')
print(training_data_set.head())
print('----- -----')
print('Testing Set')
print(testing_data_set.head())

Training Set
    action  color
0    store    red
1    store    red
2    store    red
3    store  white
4  restore    red
----- -----
Testing Set
    action  color
0    store    red
1    store  white
2  restore  white
3    store    red
4    store   blue


Now that we have imported our training and test data. 

We will calculate the distribution of the different items and actions in the training set.

In [3]:
distribution_training = training_data_set.copy()
distribution_training = distribution_training.groupby(['action', 'color']).size().reset_index(name='count')
distribution_training['count'] = distribution_training['count'].div(len(training_data_set))

distribution_training

,action,color,count
0,restore,blue,0.121683
1,restore,red,0.252415
2,restore,white,0.125841
3,store,blue,0.121683
4,store,red,0.252415
5,store,white,0.125963


# III. Markov Decision Process

The Markov decision process, better known as MDP, is an approach in reinforcement learning to take decisions in a gridworld environment. A gridworld environment consists of states in the form of grids.

Markov decision processes formally describe an environment for reinforcement learning, where the environment is fully observable ie the current state completely characterises the process

Markov property : "The future is independent of the past given the present"

Thus, any reinforcement learning task composed of a set of states, actions, and rewards that follows the Markov property would be considered an MDP.

The solution to an MDP is called a policy and the objective is to find the optimal policy for that MDP task.

A Markov Decision Process (MDP) model contains:

* A set of possible world states S.
* A set of Models.
* A set of possible actions A.
* A real valued reward function R(s,a).
* A policy the solution of Markov Decision Process.

## 1. Actions

A is a set of all possible action. A(s) defines the set of actions that can be taken being in state S.

Our robot can either retrieve or deposit objects in the warehouse. However, it should be noted that we have 3 different types of items, which will make us take different actions depending on the item chosen.

in short, our robot will be able to perform the different actions: 

* pick up or store red item --> pick_up_red, store_red
* pick up or store blue item --> pick_up_blue, store_blue
* pick up or store white item --> pick_up_white, store_white

In [4]:
all_actions = ['pick_up_red', 'pick_up_blue', 'pick_up_white', 'store_red', 'store_blue', 'store_white']

## 2. States

A State is a set of tokens that represent every state that the agent can be in.

Given our environment (warehouse 2*2) we can have different states depending on the items that would be present in the warehouse. In short, to obtain our states we must think of all the possible combinations in which our environment could be found. 

For example: at the beginning our warehouse is empty, or the warehouse already contains a red item or the warehouse already contains 2 blue items...

we can therefore see that each of the above scenarios constitutes different states of our environment. 

In [31]:
stored_item = ['empty', 'red', 'blue', 'white']
all_states = list(product(stored_item, stored_item, stored_item, stored_item, all_actions))

assert len(all_states) == 1536
print(all_states[:10:])

[('empty', 'empty', 'empty', 'empty', 'pick_up_red'), ('empty', 'empty', 'empty', 'empty', 'pick_up_blue'), ('empty', 'empty', 'empty', 'empty', 'pick_up_white'), ('empty', 'empty', 'empty', 'empty', 'store_red'), ('empty', 'empty', 'empty', 'empty', 'store_blue'), ('empty', 'empty', 'empty', 'empty', 'store_white'), ('empty', 'empty', 'empty', 'red', 'pick_up_red'), ('empty', 'empty', 'empty', 'red', 'pick_up_blue'), ('empty', 'empty', 'empty', 'red', 'pick_up_white'), ('empty', 'empty', 'empty', 'red', 'store_red')]


## 3. Model 

A model (sometimes called Transition Model) is a transition function P, which predicts the next state or the dynamics of the environnement. 

It tell us the probability distribution over next possible successor states, given the current state and the action taken by the agent.

Pa(s, s') is the transition probability matrix with the probabilities to lead from state s into another state s' within the action a

For the choice of probabilities we will base ourselves on the analysis of the training data that we carried out a little earlier. We were able to see a fairly even distribution for red and blue items and a larger distribution for red items.
Another solution would also be to start with an equiprobability for each item.

Since we have a 2*2 warehouse or a total of 4 slots, we can therefore number them from 0 to 3 in order to identify each slot.

In [101]:
slot_positions = [0, 1, 2, 3]
travel_costs=[1,2,3,4]

So we will now create all our probability transition matrices

In [30]:
def transition_probability_matrix_correction(tpm):
    for index, row_vector in enumerate(tpm):
        sum = np.sum(row_vector)
        if sum == 0:
            tpm[index, index] = 1
            continue
    tpm = tpm / tpm.sum(axis=1)[:, None]
    return tpm


def check_pick_up_action(agent_position, current_state, next_state):
    item_color_to_pick_up = current_state[len(slot_positions)]
    
    # check if the wharehouse have already this item
    if not item_color_to_pick_up in current_state[:4:]:
        return False
    
    if current_state[agent_position] == item_color_to_pick_up and next_state[agent_position] == 'empty':
        return True
    
    return False
    

def check_store_action(agent_position, current_state, next_state):
    if not 'empty' in current_state:
        return False
    
    item_color_to_store = current_state[4].split(sep='_')[-1]
    
    if current_state[agent_position] == 'empty' and next_state[agent_position] == item_color_to_store:
        return True
    
    return False


def check_transition(agent_position, current_state, next_state):
    current_action = current_state[4]
    if current_action == 'pick_up_red' or current_action == 'pick_up_blue' or current_action == 'pick_up_white':
        return check_pick_up_action(agent_position, current_state, next_state)
    else:
        return check_store_action(agent_position, current_state, next_state)

    
all_tpm = []
for agent_position in slot_positions:
    tpm = np.zeros((len(all_states), len(all_states)), dtype=np.float16)
    for i, current_state in enumerate(all_states, start=0):
        for j, next_state in enumerate(all_states, start=0):
            if check_transition(agent_position, current_state, next_state):
                next_action = next_state[4]
                if next_action == 'store_red':
                    tpm[i, j] = distribution_training.loc[(distribution_training['action'] == 'store') & (distribution_training['color'] == 'red'), 'count'].item()
                elif next_action == 'store_white':
                    tpm[i, j] = distribution_training.loc[(distribution_training['action'] == 'store') & (distribution_training['color'] == 'white'), 'count'].item()
                elif next_action == 'store_blue':
                    tpm[i, j] = distribution_training.loc[(distribution_training['action'] == 'store') & (distribution_training['color'] == 'blue'), 'count'].item()
                elif next_action == 'pick_up_red':
                    tpm[i, j] = distribution_training.loc[(distribution_training['action'] == 'restore') & (distribution_training['color'] == 'red'), 'count'].item()
                elif next_action == 'pick_up_white':
                    tpm[i, j] = distribution_training.loc[(distribution_training['action'] == 'restore') & (distribution_training['color'] == 'white'), 'count'].item()
                else:
                    tpm[i, j] = distribution_training.loc[(distribution_training['action'] == 'restore') & (distribution_training['color'] == 'blue'), 'count'].item()
        
    tpm_with_good_distribution = transition_probability_matrix_correction(tpm)
    all_tpm.append(tpm_with_good_distribution)

print(len(all_tpm))
all_tpm[:1:]

4


[array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 1.]], dtype=float16)]

## 4. Reward

A Reward Rt is a scalar feedback signal that indicates how well the agent is doing at time step t. The agent's job is to maximize the expected sum of rewards. 

A Reward is a real-valued reward function. R(s) indicates the reward for simply being in the state S. 

R(S,a) indicates the reward for being in a state S and taking an action ‘a’. 

R(S,a,S’) indicates the reward for being in a state S, taking an action ‘a’ and ending up in a state S’.

In [8]:
rewards_matrix = np.zeros((len(all_states), len(slot_positions)))
for position in slot_positions:
    for i, current_state in enumerate(all_states, start=0):
        if current_state[position] == 'empty':
            rewards_matrix[i, position] = position + 1

print(rewards_matrix)

[[1. 2. 3. 4.]
 [1. 2. 3. 4.]
 [1. 2. 3. 4.]
 ...
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## 5. Policy 

A Policy is a solution to the Markov Decision Process. A policy is a mapping from S to a. It indicates the action ‘a’ to be taken while in state S.


In [102]:
mdpResultPolicy = mdptoolbox.mdp.PolicyIteration(all_tpm, rewards_matrix, 0.3, max_iter=100)
mdpResultValue = mdptoolbox.mdp.ValueIteration(all_tpm, rewards_matrix, 0.3, max_iter=100)

"""-------- HERE ARE THE SOLUTIONS ----------------"""
mdpResultPolicy.run()
mdpResultValue.run()

print('PolicyIteration:')
print(mdpResultPolicy.policy)
print(mdpResultPolicy.V)
print(mdpResultPolicy.iter)

print('ValueIteration:')
print(mdpResultValue.policy)
print(mdpResultValue.V)
print(mdpResultValue.iter)

PolicyIteration:
(3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

# IV. Testing

We will use the solution we have just found (policy) through the training of our model and use it on the test data. 

In [103]:
def determine_new_warehouse(warehouse, agent_position, action, color):
    if action == 'store' and warehouse[agent_position] == 'empty':
        warehouse[agent_position] = color
    elif action == 'restore' and warehouse[agent_position] != 'empty':
        warehouse[agent_position] = 'empty'
    return warehouse


def state_number(warehouse):
    for index, current_state in enumerate(all_states, start=0):
        if current_state == tuple(warehouse):
            return index
    return 'failed'

def current_warehouse_state(warehouse, action, color):
    if action == 'store':
        if color == 'red':
            action='store_red'
        if color == 'white':
            action='store_white'
        if color == 'blue':
            action='store_blue'
    elif action == 'restore':
        if color == 'red':
            action='pick_up_red'
        if color == 'white':
            action='pick_up_white'
        if color == 'blue':
            action='pick_up_blue'
    if len(warehouse) == len(slot_positions):
        warehouse.append(action)
    elif len(warehouse) == len(slot_positions) + 1:
        warehouse[len(slot_positions)] = action
    return warehouse
        
def test_algorithm(policy: tuple):
    warehouse = ['empty']*len(slot_positions)

    slot_to_move = 0
    warehouse_states = []
    
    for i, row in testing_data_set.iterrows():
        action_warehouse = current_warehouse_state(warehouse, row.action, row.color)
        
        state = state_number(action_warehouse)
        
        if state != 'failed':
            warehouse_position = policy[state]
            slot_to_move += travel_costs[warehouse_position] * 2
            warehouse = determine_new_warehouse(warehouse, warehouse_position, row.action, row.color)
            warehouse_tuple = tuple(warehouse)
            warehouse_states.append(warehouse_tuple)
        else:
            print(action_warehouse, state)
            return 'failed'
    result = [slot_to_move, warehouse_states]
    
    return result

In [104]:
traveled_fields_mdpResultPolicy = test_algorithm(mdpResultPolicy.policy)
traveled_fields_mdpResultValue = test_algorithm(mdpResultValue.policy)

print("ValueIteration Robot traveled: ", traveled_fields_mdpResultValue[0])
value_iter_states = traveled_fields_mdpResultValue[1]

print("PolicyIteration Robot traveled: ", traveled_fields_mdpResultPolicy[0])
policy_iter_states = traveled_fields_mdpResultPolicy[1]
print(policy_iter_states)

ValueIteration Robot traveled:  144
PolicyIteration Robot traveled:  144
[('empty', 'empty', 'empty', 'red', 'store_red'), ('empty', 'empty', 'white', 'red', 'store_white'), ('empty', 'empty', 'white', 'red', 'pick_up_white'), ('empty', 'red', 'white', 'red', 'store_red'), ('blue', 'red', 'white', 'red', 'store_blue'), ('blue', 'red', 'white', 'red', 'store_red'), ('empty', 'red', 'white', 'red', 'pick_up_red'), ('empty', 'red', 'white', 'red', 'pick_up_red'), ('empty', 'red', 'white', 'red', 'pick_up_red'), ('red', 'red', 'white', 'red', 'store_red'), ('empty', 'red', 'white', 'red', 'pick_up_blue'), ('empty', 'red', 'white', 'red', 'pick_up_red'), ('blue', 'red', 'white', 'red', 'store_blue'), ('empty', 'red', 'white', 'red', 'pick_up_blue'), ('red', 'red', 'white', 'red', 'store_red'), ('red', 'red', 'white', 'red', 'store_red'), ('red', 'red', 'white', 'red', 'store_blue'), ('empty', 'red', 'white', 'red', 'pick_up_blue'), ('empty', 'red', 'white', 'red', 'pick_up_red'), ('red', 'r

# V. Evaluation with Greedy algorithm

In order to be able to compare and evaluate our model, we also want to implement a Greedy algorithm and use it also on our test data to better assess the quality of our results.

In [105]:
def store_in_next_empty(warehouse, color, travel_costs):
    for index, pos in enumerate(warehouse, start=0):
        if pos == 'empty':
            warehouse[index] = color
            result = [travel_costs[index] * 2, warehouse]
            return result
        
    return False

def correction_first_slot(warehouse, color, travel_costs):
    for index, pos in enumerate(warehouse, start=0):
        if pos == color:
            warehouse[index] = 'empty'
            result = [travel_costs[index]*2, warehouse]
            return result
        
    return False

def greedy_distance(travel_costs):
    warehouse = ['empty'] * len(slot_positions)
    
    distance_traveled = 0
    
    warehouse_states = []
    
    for index, row in testing_data_set.iterrows():
        warehouse_state = ''
        if row.action == 'store':
            warehouse_state = store_in_next_empty(warehouse, row.color, travel_costs)
            warehouse_tuple_store = tuple(warehouse_state[1])
            warehouse_states.append(warehouse_tuple_store)
            distance_traveled += warehouse_state[0]
        elif row.action == 'restore':
            warehouse_state = correction_first_slot(warehouse, row.color, travel_costs)
            warehouse_tuple_restore = tuple(warehouse_state[1])
            warehouse_states.append(warehouse_tuple_restore)
            distance_traveled += warehouse_state[0]
    
    result = [distance_traveled, warehouse_states]
    return result

In [106]:
traveled_fields_greedy = greedy_distance(travel_costs)

print("Greedy Robot traveled: ", traveled_fields_greedy[0])

greedy_states = traveled_fields_greedy[1]
print(greedy_states)

Greedy Robot traveled:  274
[('red', 'empty', 'empty', 'empty'), ('red', 'white', 'empty', 'empty'), ('red', 'empty', 'empty', 'empty'), ('red', 'red', 'empty', 'empty'), ('red', 'red', 'blue', 'empty'), ('red', 'red', 'blue', 'red'), ('empty', 'red', 'blue', 'red'), ('empty', 'empty', 'blue', 'red'), ('empty', 'empty', 'blue', 'empty'), ('red', 'empty', 'blue', 'empty'), ('red', 'empty', 'empty', 'empty'), ('empty', 'empty', 'empty', 'empty'), ('blue', 'empty', 'empty', 'empty'), ('empty', 'empty', 'empty', 'empty'), ('red', 'empty', 'empty', 'empty'), ('red', 'red', 'empty', 'empty'), ('red', 'red', 'blue', 'empty'), ('red', 'red', 'empty', 'empty'), ('empty', 'red', 'empty', 'empty'), ('red', 'red', 'empty', 'empty'), ('red', 'red', 'blue', 'empty'), ('red', 'red', 'empty', 'empty'), ('empty', 'red', 'empty', 'empty'), ('blue', 'red', 'empty', 'empty'), ('blue', 'red', 'white', 'empty'), ('blue', 'red', 'white', 'red'), ('blue', 'empty', 'white', 'red'), ('blue', 'empty', 'white', '

All the sources used for the realization of this article...

References :

<a href="https://towardsdatascience.com/reinforcement-learning-demystified-36c39c11ec14">Reinforcement Learning Demystified</a>

<a href="https://towardsdatascience.com/reinforcement-learning-an-introduction-to-the-concepts-applications-and-code-ced6fbfd882d">Reinforcement Learning: An Introduction to the Concepts, Applications and Code</a>

<a href="https://medium.com/coinmonks/implement-reinforcement-learning-using-markov-decision-process-tutorial-272012fdae51">Implement Reinforcement learning using Markov Decision Process</a>

<a href="http://www.cs.cmu.edu/~10601b/slides/MDP_RL.pdf">Markov Decision Process and Reinforcement Learning</a>

<a href="https://hub.packtpub.com/reinforcement-learning-mdp-markov-decision-process-tutorial/">Implement Reinforcement learning using Markov Decision Process [Tutorial]</a>

<a href="https://www.geeksforgeeks.org/markov-decision-process/">Markov Decision Process</a>

<a href="http://incompleteideas.net/book/bookdraft2017nov5.pdf">SB11 - Sutton Barto - Book - Reinforcement Learning.pdf</a>